# 05 - Branched RAG (Multi-Query Retrieval)

**Complexity:** ⭐⭐⭐

**Use Cases:** Multi-intent queries, cross-domain research, comprehensive topic exploration

**Key Feature:** Generates multiple sub-queries from user question, retrieves in parallel for better coverage.

**Example:**
```
Query: "Compare OpenAI and HuggingFace embeddings for cost and performance"

Generated sub-queries:
1. "OpenAI embeddings pricing and cost"
2. "HuggingFace embeddings performance benchmarks"
3. "Comparison of embedding providers"

→ Retrieves diverse documents covering all aspects
```

## 1. Setup

In [6]:
import sys
sys.path.append('../..')

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from shared.config import OPENAI_VECTOR_STORE_PATH, DEFAULT_MODEL
from shared.utils import load_vector_store, print_section_header, format_docs
from shared.prompts import RAG_PROMPT_TEMPLATE, MULTI_QUERY_PROMPT

print_section_header("Setup: Branched RAG")

embeddings = OpenAIEmbeddings()
vectorstore = load_vector_store(OPENAI_VECTOR_STORE_PATH, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
llm = ChatOpenAI(model=DEFAULT_MODEL, temperature=0)

print("✅ Setup complete!")


SETUP: BRANCHED RAG

✓ Loaded vector store from /Users/gianlucamazza/Workspace/notebooks/llm_rag/notebooks/advanced_architectures/../../data/vector_stores/openai_embeddings
✅ Setup complete!


## 2. Multi-Query Retriever

Uses `MultiQueryRetriever` to generate alternative queries and retrieve documents.

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
import logging

print_section_header("Multi-Query Retriever")

# Custom multi-query retriever implementation (no legacy dependencies)
class CustomMultiQueryRetriever:
    """Custom multi-query retriever that generates multiple query variations."""
    
    def __init__(self, base_retriever, llm, prompt):
        self.base_retriever = base_retriever
        self.llm = llm
        self.prompt = prompt
        self.logger = logging.getLogger("custom.multi_query")
        
    def invoke(self, query: str) -> list:
        """Retrieve documents using multiple query variations."""
        # Generate alternative queries
        query_gen_chain = self.prompt | self.llm | StrOutputParser()
        generated = query_gen_chain.invoke({"question": query})
        
        # Parse generated queries (one per line)
        alternative_queries = [q.strip() for q in generated.split('\n') if q.strip()]
        all_queries = [query] + alternative_queries[:3]  # Original + top 3 alternatives
        
        self.logger.info(f"Generated queries: {all_queries}")
        
        # Retrieve documents for each query
        all_docs = []
        for q in all_queries:
            docs = self.base_retriever.invoke(q)
            all_docs.extend(docs)
        
        # Deduplicate by content
        unique_docs = []
        seen_content = set()
        for doc in all_docs:
            content_hash = doc.page_content[:200]  # Use first 200 chars for dedup
            if content_hash not in seen_content:
                unique_docs.append(doc)
                seen_content.add(content_hash)
        
        return unique_docs

# Enable logging to see generated queries
logging.basicConfig(level=logging.WARNING)
logging.getLogger("custom.multi_query").setLevel(logging.INFO)

# Create custom multi-query retriever
multi_query_retriever = CustomMultiQueryRetriever(
    base_retriever=retriever,
    llm=llm,
    prompt=MULTI_QUERY_PROMPT
)

print("✓ Custom MultiQueryRetriever created")
print("  - Generates 3 alternative queries")
print("  - Retrieves documents for each")
print("  - Deduplicates results")
print("  - No legacy dependencies!")


MULTI-QUERY RETRIEVER

✓ Custom MultiQueryRetriever created
  - Generates 3 alternative queries
  - Retrieves documents for each
  - Deduplicates results
  - No legacy dependencies!


## 3. Test Multi-Query Retrieval

In [8]:
from shared.utils import print_results

print_section_header("Multi-Query Test")

query = "How to optimize RAG system performance?"
print(f"Original Query: '{query}'\n")

# Retrieve with multi-query
docs = multi_query_retriever.invoke(query)

print(f"\n✓ Retrieved {len(docs)} unique documents")
print_results(docs, "Multi-Query Results", max_docs=4, preview_length=150)


MULTI-QUERY TEST

Original Query: 'How to optimize RAG system performance?'



INFO:custom.multi_query:Generated queries: ['How to optimize RAG system performance?', 'What are the best practices for improving the performance of a RAG system?', 'How can I enhance the efficiency of a RAG system?', 'What strategies can be implemented to optimize the performance of a RAG system?']



✓ Retrieved 2 unique documents

Multi-Query Results
--------------------------------------------------------------------------------

1. Source: https://python.langchain.com/docs/use_cases/question_answering/
   Type: web_documentation
   Date: 2025-11-12
   Content: We can see the full sequence of steps, along with latency and other metadata, in the LangSmith trace.
You can add a deeper level of control and custom...

2. Source: https://python.langchain.com/docs/use_cases/question_answering/
   Type: web_documentation
   Date: 2025-11-12
   Content: A RAG agent that executes searches with a simple tool. This is a good general-purpose implementation.
A two-step RAG chain that uses just a single LLM...


## 4. Build Branched RAG Chain

In [9]:
print_section_header("Branched RAG Chain")

def multi_query_retrieve(query: str):
    """Wrapper function for multi-query retrieval."""
    return multi_query_retriever.invoke(query)

branched_chain = (
    {"context": RunnableLambda(multi_query_retrieve) | format_docs, "input": RunnablePassthrough()}
    | RAG_PROMPT_TEMPLATE
    | llm
    | StrOutputParser()
)

print("✓ Branched RAG chain created")

# Test
query = "What are the differences between similarity and MMR retrieval?"
print(f"\nQuery: '{query}'\n")
print("=" * 80)

response = branched_chain.invoke(query)
print(response)

print("\n" + "=" * 80)
print("\n✅ Branched RAG provides more comprehensive answers!")


BRANCHED RAG CHAIN

✓ Branched RAG chain created

Query: 'What are the differences between similarity and MMR retrieval?'



INFO:custom.multi_query:Generated queries: ['What are the differences between similarity and MMR retrieval?', 'What distinguishes similarity retrieval from MMR retrieval?', 'How do similarity retrieval and MMR retrieval differ from each other?', 'Can you explain the differences between similarity-based retrieval and MMR retrieval?']


The context provided does not contain any information about the differences between similarity and MMR (Maximal Marginal Relevance) retrieval. Therefore, I cannot answer your question based on the given context.


✅ Branched RAG provides more comprehensive answers!


## 5. Comparison: Simple vs Branched

In [10]:
print_section_header("Comparison")

# Simple RAG
simple_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | RAG_PROMPT_TEMPLATE
    | llm
    | StrOutputParser()
)

query = "How to implement embeddings in RAG?"

print(f"Query: '{query}'\n")
print("[SIMPLE RAG]")
simple_docs = retriever.invoke(query)
print(f"Documents retrieved: {len(simple_docs)}")

print("\n[BRANCHED RAG]")
branched_docs = multi_query_retriever.invoke(query)
print(f"Documents retrieved: {len(branched_docs)} (from multiple queries)")

print("\n💡 Branched RAG typically retrieves more diverse documents")


COMPARISON

Query: 'How to implement embeddings in RAG?'

[SIMPLE RAG]
Documents retrieved: 4

[BRANCHED RAG]
Documents retrieved: 4

[BRANCHED RAG]


INFO:custom.multi_query:Generated queries: ['How to implement embeddings in RAG?', 'What are the steps to implement embeddings in a Retrieval-Augmented Generation (RAG) model?', 'Can you explain the process of integrating embeddings into a RAG framework?', 'What methods can be used to apply embeddings in a Retrieval-Augmented Generation system?']


Documents retrieved: 5 (from multiple queries)

💡 Branched RAG typically retrieves more diverse documents


## Summary

**Flow:**
```
User Query → Generate Sub-Queries → Parallel Retrieval → Deduplicate → LLM → Response
```

**Advantages:**
✅ Better coverage for complex queries  
✅ Captures multiple aspects  
✅ More diverse perspectives  
✅ Handles multi-intent questions  

**Limitations:**
- Higher latency (multiple retrievals)
- More API calls (cost)
- May retrieve redundant info

**When to Use:**
- Broad research questions
- Multi-concept queries
- Comprehensive coverage needed

**Next:** [06_hyde.ipynb](06_hyde.ipynb) - Hypothetical Document Embeddings